## A workbook companion to the introduction to the DataFrames.jl package

[Bogumił Kamiński](http://bogumilkaminski.pl/about/)

## Getting ready

We will use [this](http://archive.ics.uci.edu/ml/datasets/communities+and+crime) dataset. So first we download the required files.

In [1]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data", "data.txt")

"data.txt"

In [2]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names", "meta.txt")

"meta.txt"

In [3]:
using DataFrames
using CSV
using FreqTables
using Plots, StatsPlots
using GLM
using Statistics

You could also consider installing https://github.com/JuliaStats/StatsKit.jl package, but we want to be explicit what we use.

## Loading the data

Here we have to do some non-standard operations.

In [4]:
colnamelines = filter(line -> startswith(line, "@attribute"), readlines("meta.txt"))

128-element Array{String,1}:
 "@attribute state numeric"              
 "@attribute county numeric"             
 "@attribute community numeric"          
 "@attribute communityname string"       
 "@attribute fold numeric"               
 "@attribute population numeric"         
 "@attribute householdsize numeric"      
 "@attribute racepctblack numeric"       
 "@attribute racePctWhite numeric"       
 "@attribute racePctAsian numeric"       
 ⋮                                       
 "@attribute PopDens numeric"            
 "@attribute PctUsePubTrans numeric"     
 "@attribute PolicCars numeric"          
 "@attribute PolicOperBudg numeric"      
 "@attribute LemasPctPolicOnPatr numeric"
 "@attribute LemasGangUnitDeploy numeric"
 "@attribute LemasPctOfficDrugUn numeric"
 "@attribute PolicBudgPerPop numeric"    
 "@attribute ViolentCrimesPerPop numeric"

In [5]:
colnames = Symbol.(getindex.(split.(colnamelines), 2))

128-element Array{Symbol,1}:
 :state              
 :county             
 :community          
 :communityname      
 :fold               
 :population         
 :householdsize      
 :racepctblack       
 :racePctWhite       
 :racePctAsian       
 ⋮                   
 :PopDens            
 :PctUsePubTrans     
 :PolicCars          
 :PolicOperBudg      
 :LemasPctPolicOnPatr
 :LemasGangUnitDeploy
 :LemasPctOfficDrugUn
 :PolicBudgPerPop    
 :ViolentCrimesPerPop

In [6]:
coltypes = getindex.(split.(colnamelines), 3)

128-element Array{SubString{String},1}:
 "numeric"
 "numeric"
 "numeric"
 "string" 
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 ⋮        
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"
 "numeric"

In [7]:
df = CSV.File("data.txt", header=colnames) |> DataFrame

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup
,Int64,String,String,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.9,0.12,0.17,0.34,0.47,0.29,0.32,0.2,1.0,0.37,0.72,0.34,0.6,0.29,0.15,0.43,0.39,0.4,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.1,0.18,0.48,0.27,0.68,0.23,0.41,0.25,0.52,0.68,0.4,0.75,0.75,0.35,0.55,0.59,0.61,0.56,0.74,0.76,0.04,0.14,0.03,0.24,0.27,0.37,0.39,0.07,0.07,0.08,0.08,0.89,0.06,0.14,0.13,0.33,0.39,0.28,0.55
2,53,?,?,Tukwilacity,1,0.0,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.3,0.22,0.35,0.01,0.24,0.14,0.24,0.3,0.27,0.73,0.57,0.15,0.42,0.36,1.0,0.63,0.91,1.0,0.29,0.43,0.47,0.6,0.39,0.46,0.53,0.0,0.24,0.01,0.52,0.62,0.64,0.63,0.25,0.27,0.25,0.23,0.84,0.1,0.16,0.1,0.17,0.29,0.17,0.26
3,24,?,?,Aberdeentown,1,0.0,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.0,0.0,0.3,0.58,0.19,0.39,0.38,0.4,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,0.29,0.49,0.32,0.63,0.41,0.71,0.7,0.45,0.42,0.44,0.43,0.43,0.71,0.67,0.01,0.46,0.0,0.07,0.06,0.15,0.19,0.02,0.02,0.04,0.05,0.88,0.04,0.2,0.2,0.46,0.52,0.43,0.42
4,34,5,81440,Willingborotownship,1,0.04,0.77,1.0,0.08,0.12,0.1,0.51,0.5,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.2,0.82,0.51,0.36,0.4,0.39,0.16,0.25,0.36,0.44,0.01,0.1,0.09,0.25,0.31,0.33,0.71,0.36,0.45,0.37,0.39,0.34,0.45,0.49,0.44,0.75,0.65,0.54,0.83,0.65,0.85,0.86,0.03,0.33,0.02,0.11,0.2,0.3,0.31,0.05,0.08,0.11,0.11,0.81,0.08,0.56,0.62,0.85,0.77,1.0,0.94
5,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.5,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.0,0.74,0.51,0.48,0.0,0.06,0.25,0.3,0.33,0.12,0.65,0.67,0.38,0.42,0.46,0.22,0.27,0.2,0.21,0.51,0.91,0.91,0.89,0.85,0.4,0.6,0.0,0.06,0.0,0.03,0.07,0.2,0.27,0.01,0.02,0.04,0.05,0.88,0.05,0.16,0.19,0.59,0.6,0.37,0.89
6,6,?,?,SouthPasadenacity,1,0.02,0.28,0.06,0.54,1.0,0.25,0.31,0.48,0.27,0.37,0.04,1.0,0.52,0.68,0.2,0.61,0.28,0.15,0.25,0.62,0.72,0.76,0.77,0.28,0.52,0.48,0.6,0.01,0.12,0.13,0.12,0.8,0.1,0.65,0.19,0.77,0.06,0.91,0.49,0.57,0.61,0.58,0.44,0.62,0.69,0.87,0.53,0.3,0.43,0.0,0.11,0.04,0.3,0.35,0.43,0.47,0.5,0.5,0.56,0.57,0.45,0.28,0.25,0.19,0.29,0.53,0.18,0.39
7,44,7,41500,Lincolntown,1,0.01,0.39,0.0,0.98,0.06,0.02,0.3,0.37,0.23,0.6,0.02,0.81,0.42,0.5,0.23,0.68,0.61,0.21,0.54,0.43,0.47,0.44,0.4,0.24,0.86,0.24,0.36,0.01,0.11,0.29,0.41,0.36,0.28,0.54,0.44,0.53,0.33,0.49,0.25,0.34,0.28,0.28,0.42,0.77,0.81,0.79,0.74,0.57,0.62,0.0,0.13,0.01,0.0,0.02,0.02,0.1,0.0,0.01,0.01,0.03,0.73,0.05,0.12,0.13,0.42,0.

We see that theere is a problem that `?` indicates a missing value in the data set

In [8]:
df = CSV.File("data.txt", header=colnames, missingstring="?") |> DataFrame

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup
,Int64,Int64⍰,Int64⍰,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,8,missing,missing,Lakewoodcity,1,0.19,0.33,0.02,0.9,0.12,0.17,0.34,0.47,0.29,0.32,0.2,1.0,0.37,0.72,0.34,0.6,0.29,0.15,0.43,0.39,0.4,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.1,0.18,0.48,0.27,0.68,0.23,0.41,0.25,0.52,0.68,0.4,0.75,0.75,0.35,0.55,0.59,0.61,0.56,0.74,0.76,0.04,0.14,0.03,0.24,0.27,0.37,0.39,0.07,0.07,0.08,0.08,0.89,0.06,0.14,0.13,0.33,0.39,0.28,0.55
2,53,missing,missing,Tukwilacity,1,0.0,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.3,0.22,0.35,0.01,0.24,0.14,0.24,0.3,0.27,0.73,0.57,0.15,0.42,0.36,1.0,0.63,0.91,1.0,0.29,0.43,0.47,0.6,0.39,0.46,0.53,0.0,0.24,0.01,0.52,0.62,0.64,0.63,0.25,0.27,0.25,0.23,0.84,0.1,0.16,0.1,0.17,0.29,0.17,0.26
3,24,missing,missing,Aberdeentown,1,0.0,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.0,0.0,0.3,0.58,0.19,0.39,0.38,0.4,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,0.29,0.49,0.32,0.63,0.41,0.71,0.7,0.45,0.42,0.44,0.43,0.43,0.71,0.67,0.01,0.46,0.0,0.07,0.06,0.15,0.19,0.02,0.02,0.04,0.05,0.88,0.04,0.2,0.2,0.46,0.52,0.43,0.42
4,34,5,81440,Willingborotownship,1,0.04,0.77,1.0,0.08,0.12,0.1,0.51,0.5,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.2,0.82,0.51,0.36,0.4,0.39,0.16,0.25,0.36,0.44,0.01,0.1,0.09,0.25,0.31,0.33,0.71,0.36,0.45,0.37,0.39,0.34,0.45,0.49,0.44,0.75,0.65,0.54,0.83,0.65,0.85,0.86,0.03,0.33,0.02,0.11,0.2,0.3,0.31,0.05,0.08,0.11,0.11,0.81,0.08,0.56,0.62,0.85,0.77,1.0,0.94
5,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.5,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.0,0.74,0.51,0.48,0.0,0.06,0.25,0.3,0.33,0.12,0.65,0.67,0.38,0.42,0.46,0.22,0.27,0.2,0.21,0.51,0.91,0.91,0.89,0.85,0.4,0.6,0.0,0.06,0.0,0.03,0.07,0.2,0.27,0.01,0.02,0.04,0.05,0.88,0.05,0.16,0.19,0.59,0.6,0.37,0.89
6,6,missing,missing,SouthPasadenacity,1,0.02,0.28,0.06,0.54,1.0,0.25,0.31,0.48,0.27,0.37,0.04,1.0,0.52,0.68,0.2,0.61,0.28,0.15,0.25,0.62,0.72,0.76,0.77,0.28,0.52,0.48,0.6,0.01,0.12,0.13,0.12,0.8,0.1,0.65,0.19,0.77,0.06,0.91,0.49,0.57,0.61,0.58,0.44,0.62,0.69,0.87,0.53,0.3,0.43,0.0,0.11,0.04,0.3,0.35,0.43,0.47,0.5,0.5,0.56,0.57,0.45,0.28,0.25,0.19,0.29,0.53,0.18,0.39
7,44,7,41500,Lincolntown,1,0.01,0.39,0.0,0.98,0.06,0.02,0.3,0.37,0.23,0.6,0.02,0.81,0.42,0.5,0.23,0.68,0.61,0.21,0.54,0.43,0.47,0.44,0.4,0.24,0.86,0.24,0.36,0.01,0.11,0.29,0.41,0.36,0.28,0.54,0.44,0.53,0.33,0.49,0.25,0.34,0.28,0.28,0.42,0.77,0.81,0.79,0.74,0.57,0.62,0.0,0.13,0.01,0.0,0.02,0.02,

Let us check if what we detected is consistent with the metadata

In [9]:
freqtable(eltypes(df), coltypes)

5×2 Named Array{Int64,2}
            Dim1 ╲ Dim2 │ "numeric"   "string"
────────────────────────┼─────────────────────
Union{Missing, Float64} │        23          0
Union{Missing, Int64}   │         2          0
String                  │         0          1
Float64                 │       100          0
Int64                   │         2          0

In [10]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,DataType
1,state,28.6836,1,34.0,56,,,Int64
2,county,58.8268,1,23.0,840,,1174,Int64
3,community,46188.3,70,48090.0,94597,,1177,Int64
4,communityname,,Aberdeencity,,Zanesvillecity,1828,,String
5,fold,5.49398,1,5.0,10,,,Int64
6,population,0.0575928,0.0,0.02,1.0,,,Float64
7,householdsize,0.463395,0.0,0.44,1.0,,,Float64
8,racepctblack,0.179629,0.0,0.06,1.0,,,Float64
9,racePctWhite,0.753716,0.0,0.85,1.0,,,Float64


We can "force" Julia to print all rows of a data frame in an unformatted manner like this. Use this with caution on large data frames.

In [11]:
show(describe(df), allrows=true)

128×8 DataFrame
│ Row │ variable              │ mean      │ min          │ median  │ max            │ nunique │ nmissing │ eltype   │
│     │ Symbol                │ Union…    │ Any          │ Union…  │ Any            │ Union…  │ Union…   │ DataType │
├─────┼───────────────────────┼───────────┼──────────────┼─────────┼────────────────┼─────────┼──────────┼──────────┤
│ 1   │ state                 │ 28.6836   │ 1            │ 34.0    │ 56             │         │          │ Int64    │
│ 2   │ county                │ 58.8268   │ 1            │ 23.0    │ 840            │         │ 1174     │ Int64    │
│ 3   │ community             │ 46188.3   │ 70           │ 48090.0 │ 94597          │         │ 1177     │ Int64    │
│ 4   │ communityname         │           │ Aberdeencity │         │ Zanesvillecity │ 1828    │          │ String   │
│ 5   │ fold                  │ 5.49398   │ 1            │ 5.0     │ 10             │         │          │ Int64    │
│ 6   │ population            │ 0.057592

│ 67  │ PctNotSpeakEnglWell   │ 0.150587  │ 0.0          │ 0.06    │ 1.0            │         │          │ Float64  │
│ 68  │ PctLargHouseFam       │ 0.267608  │ 0.0          │ 0.2     │ 1.0            │         │          │ Float64  │
│ 69  │ PctLargHouseOccup     │ 0.251891  │ 0.0          │ 0.19    │ 1.0            │         │          │ Float64  │
│ 70  │ PersPerOccupHous      │ 0.462101  │ 0.0          │ 0.44    │ 1.0            │         │          │ Float64  │
│ 71  │ PersPerOwnOccHous     │ 0.494428  │ 0.0          │ 0.48    │ 1.0            │         │          │ Float64  │
│ 72  │ PersPerRentOccHous    │ 0.404097  │ 0.0          │ 0.36    │ 1.0            │         │          │ Float64  │
│ 73  │ PctPersOwnOccup       │ 0.562598  │ 0.0          │ 0.56    │ 1.0            │         │          │ Float64  │
│ 74  │ PctPersDenseHous      │ 0.186264  │ 0.0          │ 0.11    │ 1.0            │         │          │ Float64  │
│ 75  │ PctHousLess3BR        │ 0.495186  │ 0.0         

Similarly we can make Julia to use "paging mode" when showing all columns. Again - showing all rows in ths mode is not advisable.

In [12]:
show(IOContext(stdout, :displaysize=>(20,100)), first(df, 5), splitcols=true)

5×128 DataFrame
│ Row │ state │ county  │ community │ communityname       │ fold  │ population │ householdsize │
│     │ Int64 │ Int64⍰  │ Int64⍰    │ String              │ Int64 │ Float64    │ Float64       │
├─────┼───────┼─────────┼───────────┼─────────────────────┼───────┼────────────┼───────────────┤
│ 1   │ 8     │ missing │ missing   │ Lakewoodcity        │ 1     │ 0.19       │ 0.33          │
│ 2   │ 53    │ missing │ missing   │ Tukwilacity         │ 1     │ 0.0        │ 0.16          │
│ 3   │ 24    │ missing │ missing   │ Aberdeentown        │ 1     │ 0.0        │ 0.42          │
│ 4   │ 34    │ 5       │ 81440     │ Willingborotownship │ 1     │ 0.04       │ 0.77          │
│ 5   │ 42    │ 95      │ 6096      │ Bethlehemtownship   │ 1     │ 0.01       │ 0.55          │

│ Row │ racepctblack │ racePctWhite │ racePctAsian │ racePctHisp │ agePct12t21 │ agePct12t29 │
│     │ Float64      │ Float64      │ Float64      │ Float64     │ Float64     │ Float64     │
├─────┼──────────

│ 4   │ 0.08         │ 0.11         │ 0.11          │ 0.81             │ 0.08                │
│ 5   │ 0.02         │ 0.04         │ 0.05          │ 0.88             │ 0.05                │

│ Row │ PctLargHouseFam │ PctLargHouseOccup │ PersPerOccupHous │ PersPerOwnOccHous │
│     │ Float64         │ Float64           │ Float64          │ Float64           │
├─────┼─────────────────┼───────────────────┼──────────────────┼───────────────────┤
│ 1   │ 0.14            │ 0.13              │ 0.33             │ 0.39              │
│ 2   │ 0.16            │ 0.1               │ 0.17             │ 0.29              │
│ 3   │ 0.2             │ 0.2               │ 0.46             │ 0.52              │
│ 4   │ 0.56            │ 0.62              │ 0.85             │ 0.77              │
│ 5   │ 0.16            │ 0.19              │ 0.59             │ 0.6               │

│ Row │ PersPerRentOccHous │ PctPersOwnOccup │ PctPersDenseHous │ PctHousLess3BR │ MedNumBR │
│     │ Float64            │ Float

│     │ Float64⍰      │ Float64⍰            │ Float64⍰            │ Float64             │
├─────┼───────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ 1   │ 0.04          │ 0.9                 │ 0.5                 │ 0.32                │
│ 2   │ missing       │ missing             │ missing             │ 0.0                 │
│ 3   │ missing       │ missing             │ missing             │ 0.0                 │
│ 4   │ missing       │ missing             │ missing             │ 0.0                 │
│ 5   │ missing       │ missing             │ missing             │ 0.0                 │

│ Row │ PolicBudgPerPop │ ViolentCrimesPerPop │
│     │ Float64⍰        │ Float64             │
├─────┼─────────────────┼─────────────────────┤
│ 1   │ 0.14            │ 0.2                 │
│ 2   │ missing         │ 0.67                │
│ 3   │ missing         │ 0.43                │
│ 4   │ missing         │ 0.12                │
│ 5   │ missing         │ 0.03   

## Analyzing the data

We will not want to use the first five columns from the data set. Let us get rid of them. This will be an in-place operation.

In [13]:
select!(df, Not(1:5))

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.19,0.33,0.02,0.9,0.12,0.17,0.34,0.47,0.29,0.32,0.2,1.0,0.37,0.72,0.34,0.6,0.29,0.15,0.43,0.39,0.4,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.1,0.18,0.48,0.27,0.68,0.23,0.41,0.25,0.52,0.68,0.4,0.75,0.75,0.35,0.55,0.59,0.61,0.56,0.74,0.76,0.04,0.14,0.03,0.24,0.27,0.37,0.39,0.07,0.07,0.08,0.08,0.89,0.06,0.14,0.13,0.33,0.39,0.28,0.55,0.09,0.51,0.5
2,0.0,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.3,0.22,0.35,0.01,0.24,0.14,0.24,0.3,0.27,0.73,0.57,0.15,0.42,0.36,1.0,0.63,0.91,1.0,0.29,0.43,0.47,0.6,0.39,0.46,0.53,0.0,0.24,0.01,0.52,0.62,0.64,0.63,0.25,0.27,0.25,0.23,0.84,0.1,0.16,0.1,0.17,0.29,0.17,0.26,0.2,0.82,0.0
3,0.0,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.0,0.0,0.3,0.58,0.19,0.39,0.38,0.4,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,0.29,0.49,0.32,0.63,0.41,0.71,0.7,0.45,0.42,0.44,0.43,0.43,0.71,0.67,0.01,0.46,0.0,0.07,0.06,0.15,0.19,0.02,0.02,0.04,0.05,0.88,0.04,0.2,0.2,0.46,0.52,0.43,0.42,0.15,0.51,0.5
4,0.04,0.77,1.0,0.08,0.12,0.1,0.51,0.5,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.2,0.82,0.51,0.36,0.4,0.39,0.16,0.25,0.36,0.44,0.01,0.1,0.09,0.25,0.31,0.33,0.71,0.36,0.45,0.37,0.39,0.34,0.45,0.49,0.44,0.75,0.65,0.54,0.83,0.65,0.85,0.86,0.03,0.33,0.02,0.11,0.2,0.3,0.31,0.05,0.08,0.11,0.11,0.81,0.08,0.56,0.62,0.85,0.77,1.0,0.94,0.12,0.01,0.5
5,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,0.02,0.9,0.5,0.72,0.16,0.68,0.44,0.11,0.71,0.46,0.43,0.41,0.28,0.0,0.74,0.51,0.48,0.0,0.06,0.25,0.3,0.33,0.12,0.65,0.67,0.38,0.42,0.46,0.22,0.27,0.2,0.21,0.51,0.91,0.91,0.89,0.85,0.4,0.6,0.0,0.06,0.0,0.03,0.07,0.2,0.27,0.01,0.02,0.04,0.05,0.88,0.05,0.16,0.19,0.59,0.6,0.37,0.89,0.02,0.19,0.5
6,0.02,0.28,0.06,0.54,1.0,0.25,0.31,0.48,0.27,0.37,0.04,1.0,0.52,0.68,0.2,0.61,0.28,0.15,0.25,0.62,0.72,0.76,0.77,0.28,0.52,0.48,0.6,0.01,0.12,0.13,0.12,0.8,0.1,0.65,0.19,0.77,0.06,0.91,0.49,0.57,0.61,0.58,0.44,0.62,0.69,0.87,0.53,0.3,0.43,0.0,0.11,0.04,0.3,0.35,0.43,0.47,0.5,0.5,0.56,0.57,0.45,0.28,0.25,0.19,0.29,0.53,0.18,0.39,0.26,0.73,0.0
7,0.01,0.39,0.0,0.98,0.06,0.02,0.3,0.37,0.23,0.6,0.02,0.81,0.42,0.5,0.23,0.68,0.61,0.21,0.54,0.43,0.47,0.44,0.4,0.24,0.86,0.24,0.36,0.01,0.11,0.29,0.41,0.36,0.28,0.54,0.44,0.53,0.33,0.49,0.25,0.34,0.28,0.28,0.42,0.77,0.81,0.79,0.74,0.57,0.62,0.0,0.13,0.01,0.0,0.02,0.02,0.1,0.0,0.01,0.01,0.03,0.73,0.05,0.12,0.13,0.42,0.54,0.24,0.65,0.03,0.46,0.5
8,0.01,0.74,0.03,0.46,0.2,1.0,0.52,0.55,0.36,0.35,0.0,0.0,0.16,0.44,1.0,0.23,

Analyzing the correlation between `:ViolentCrimesPerProp` and other variables.

The challenge is that some of the variables have missing data and other do not have them. We want to use pairwise complete observations approach (delete only what we have to delete when calculating the correlation).

This will be our fisrst encounter with views of a `DataFrame`. The use of `@view` avoids copying of the data. This improves the performance, but in general you should not mutate the view, as the changes will propagate back to the parent.

In [14]:
results = DataFrame()
for col in names(df)[1:end-1]
    cols = [col, :ViolentCrimesPerPop]
    subdf = @view df[completecases(df, cols), cols]
    push!(results, (vairable=col, correlation=cor(subdf[!, 1], subdf[!, 2])))
end

In [15]:
results

,vairable,correlation
,Symbol,Float64
1,population,0.367157
2,householdsize,-0.0349226
3,racepctblack,0.631264
4,racePctWhite,-0.68477
5,racePctAsian,0.0376218
6,racePctHisp,0.29305
7,agePct12t21,0.0604772
8,agePct12t29,0.153357
9,agePct16t24,0.0993467


What if we want to see the most correlated values on top of the data frame?

In [16]:
sort!(results, :correlation, by = abs, rev=true)

,vairable,correlation
,Symbol,Float64
1,PctKids2Par,-0.738424
2,PctIlleg,0.737957
3,PctFam2Par,-0.706667
4,racePctWhite,-0.68477
5,PctYoungKids2Par,-0.666059
6,PctTeen2Par,-0.661582
7,racepctblack,0.631264
8,pctWInvInc,-0.576324
9,pctWPubAsst,0.574665


Now let us do some additional analyses.

In [17]:
@df select(df, [results[1:4, 1]; :ViolentCrimesPerPop]) corrplot(cols(1:5))

In [18]:
lm(@formula(ViolentCrimesPerPop ~ PctKids2Par + PctIlleg + racePctWhite), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

ViolentCrimesPerPop ~ 1 + PctKids2Par + PctIlleg + racePctWhite

Coefficients:
─────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error   t value  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)    0.649797   0.0343578   18.9126    <1e-72   0.582416   0.717178
PctKids2Par   -0.444269   0.0322473  -13.7769    <1e-40  -0.507511  -0.381027
PctIlleg       0.192406   0.0346147    5.5585    <1e-7    0.124521   0.260291
racePctWhite  -0.244363   0.0225653  -10.8291    <1e-25  -0.288617  -0.200108
─────────────────────────────────────────────────────────────────────────────

Now feel free to experiment with the data set.